Note: Run this before running Process-TAZ-Outputs-For-Web

In [25]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.geometry import Geometry
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd
import glob
import shutil
import zipfile

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [26]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [28]:
# read inputs
parcel_se_folder = r'\\server1\Volumef\SHARED\Josh\REMM Runs\base_2019_parcel_se_20230127'
parcel_se_folder_2015 = r'\\server1\Volumef\SHARED\Josh\REMM Runs\base_2015_parcel_se_20230223'

eq = pd.read_csv(r".\Inputs\parcel_eq_v5.csv")
eq_2015 = pd.read_csv(r".\Inputs\parcel_eq_2015_v1.csv")


centers_sdf = pd.DataFrame.spatial.from_featureclass(r".\Inputs\WC_2050_Centers.shp")[['CenterName', 'AreaType', 'CO_NAME', 'ACRES', 'DEVACRES', 'SHAPE']].copy()
centers_sdf = centers_sdf[centers_sdf['CO_NAME'] != 'BOX ELDER'].copy()
centers_sdf.rename({'AreaType':'CenterType'}, inplace=True, axis=1)
centers_sdf['CenterName'] = centers_sdf['CenterName'].str.replace("/", " ")
centers_sdf['CenterName'] = centers_sdf['CenterName'].str.replace(".", "")
centers_sdf['CenterName'] = centers_sdf['CenterName'].str.replace("'", "")

<ipython-input-28-d5aa31b69006>:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  centers_sdf['CenterName'] = centers_sdf['CenterName'].str.replace(".", "")


In [29]:
# get list of se pkls
parcel_se = glob.glob(os.path.join(parcel_se_folder,'averaged_parcel_se_*.pkl'))
parcel_se_2015 = glob.glob(os.path.join(parcel_se_folder_2015,'averaged_parcel_se_b2015_*.pkl'))[4:] # start from 2019

# Parcel Equivalency Table
centers_eq_ids = eq[eq['CENTER_NAME'] != 'Non-center']['parcel_id'].to_list()
centers_eq_ids_2015 = eq_2015[eq_2015['CENTER_NAME'] != 'Non-center']['parcel_id'].to_list()


centers_lu = eq[['parcel_id', 'CENTER_NAME']].copy()
centers_lu['CENTER_NAME'] = centers_lu['CENTER_NAME'].str.replace("/", " ")
centers_lu['CENTER_NAME'] = centers_lu['CENTER_NAME'].str.replace(".", "")
centers_lu['CENTER_NAME'] = centers_lu['CENTER_NAME'].str.replace("'", "")

centers_lu_2015 = eq_2015[['parcel_id', 'CENTER_NAME']].copy()
centers_lu_2015['CENTER_NAME'] = centers_lu_2015['CENTER_NAME'].str.replace("/", " ")
centers_lu_2015['CENTER_NAME'] = centers_lu_2015['CENTER_NAME'].str.replace(".", "")
centers_lu_2015['CENTER_NAME'] = centers_lu_2015['CENTER_NAME'].str.replace("'", "")

<ipython-input-29-f8a2046e2c19>:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  centers_lu['CENTER_NAME'] = centers_lu['CENTER_NAME'].str.replace(".", "")
<ipython-input-29-f8a2046e2c19>:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  centers_lu_2015['CENTER_NAME'] = centers_lu_2015['CENTER_NAME'].str.replace(".", "")


In [30]:
centers_lu_2015[centers_lu_2015['CENTER_NAME'].str.contains("RSL") == True ]

,parcel_id,CENTER_NAME
749661,790758,SLCC RSL
749823,790920,SLCC RSL
749824,790921,SLCC RSL
749825,790922,SLCC RSL
749826,790923,SLCC RSL
...,...,...
750887,791984,SLCC RSL
750980,792077,SLCC RSL
750981,792078,SLCC RSL
750982,792079,SLCC RSL


In [31]:
outputs = r'.\Outputs'
if not os.path.exists(outputs):
    os.makedirs(outputs)

# Create folders for deliverables
map_folder = os.path.join(outputs, "map")
if not os.path.exists(map_folder):
    os.makedirs(map_folder)

chart_folder = os.path.join(outputs, "chart")
if not os.path.exists(chart_folder):
    os.makedirs(chart_folder)

In [32]:
base = centers_sdf[['CenterName']].copy()

for pkl in parcel_se:

    year = os.path.splitext(os.path.basename(pkl))[0].split('_')[-1]
    df = pd.read_pickle(pkl)
    
    df['TOTHH'] = df['households']
    df['HHPOP'] = df['hhpop'] 
    df['TPCL'] = df['total_jobs']
    df['ALLEMP'] = df['total_jobs'] # missing mining, home-based, construction 
    df['RETEMP'] = df['retail_jobs']
    df['INDEMP'] = df['industrial_jobs']
    df['OTHEMP'] = df['office_jobs']
    df['HJI'] = df['TOTHH']*1.8 + df['TPCL']
    df = df[['parcel_id', 'TOTHH', 'HHPOP', 'ALLEMP','RETEMP', 'INDEMP', 'OTHEMP', 'TPCL', 'HJI']].copy()
    df.columns = ['parcel_id', f'HH_{year}', f'POP_{year}', f'AEMP_{year}', f'RTL_{year}', f'IND_{year}', f'OTHR_{year}', f'TPCL_{year}', f'HJI_{year}']

    df= df.merge(centers_lu, on='parcel_id', how='left')
    df_by_center = df.groupby('CENTER_NAME').sum().reset_index()
    df_by_center.rename({'CENTER_NAME':'CenterName'}, inplace=True, axis=1)
    del df_by_center['parcel_id']
    base = base.merge(df_by_center, on='CenterName', how='left').copy()

se_by_year_base_2019 = base
se_by_year_base_2019 = se_by_year_base_2019.fillna(0)
se_by_year_base_2019 = se_by_year_base_2019.round()

In [33]:
se_by_year_base_2019[se_by_year_base_2019['CenterName'].str.contains("RSL") == True ]

,CenterName,HH_2019,POP_2019,AEMP_2019,RTL_2019,IND_2019,OTHR_2019,TPCL_2019,HJI_2019,HH_2020,POP_2020,AEMP_2020,RTL_2020,IND_2020,OTHR_2020,TPCL_2020,HJI_2020,HH_2021,POP_2021,AEMP_2021,RTL_2021,IND_2021,OTHR_2021,TPCL_2021,HJI_2021,HH_2022,POP_2022,AEMP_2022,RTL_2022,IND_2022,OTHR_2022,TPCL_2022,HJI_2022,HH_2023,POP_2023,AEMP_2023,RTL_2023,IND_2023,OTHR_2023,TPCL_2023,HJI_2023,HH_2024,POP_2024,AEMP_2024,RTL_2024,IND_2024,OTHR_2024,TPCL_2024,HJI_2024,HH_2025,POP_2025,AEMP_2025,RTL_2025,IND_2025,OTHR_2025,TPCL_2025,HJI_2025,HH_2026,POP_2026,AEMP_2026,RTL_2026,IND_2026,OTHR_2026,TPCL_2026,HJI_2026,HH_2027,POP_2027,AEMP_2027,RTL_2027,IND_2027,OTHR_2027,TPCL_2027,HJI_2027,HH_2028,POP_2028,AEMP_2028,RTL_2028,IND_2028,OTHR_2028,TPCL_2028,HJI_2028,HH_2029,POP_2029,AEMP_2029,RTL_2029,IND_2029,OTHR_2029,TPCL_2029,HJI_2029,HH_2030,POP_2030,AEMP_2030,RTL_2030,IND_2030,OTHR_2030,TPCL_2030,HJI_2030,HH_2031,POP_2031,AEMP_2031,RTL_2031,IND_2031,OTHR_2031,TPCL_2031,HJI_2031,HH_2032,POP_2032,AEMP_2032,RTL_2032,IND_2032,OTHR_2032,TPCL_2032,HJI_2032,HH_2033,POP_2033,AEMP_2033,RTL_2033,IND_2033,OTHR_2033,TPCL_2033,HJI_2033,HH_2034,POP_2034,AEMP_2034,RTL_2034,IND_2034,OTHR_2034,TPCL_2034,HJI_2034,HH_2035,POP_2035,AEMP_2035,RTL_2035,IND_2035,OTHR_2035,TPCL_2035,HJI_2035,HH_2036,POP_2036,AEMP_2036,RTL_2036,IND_2036,OTHR_2036,TPCL_2036,HJI_2036,HH_2037,POP_2037,AEMP_2037,RTL_2037,IND_2037,OTHR_2037,TPCL_2037,HJI_2037,HH_2038,POP_2038,AEMP_2038,RTL_2038,IND_2038,OTHR_2038,TPCL_2038,HJI_2038,HH_2039,POP_2039,AEMP_2039,RTL_2039,IND_2039,OTHR_2039,TPCL_2039,HJI_2039,HH_2040,POP_2040,AEMP_2040,RTL_2040,IND_2040,OTHR_2040,TPCL_2040,HJI_2040,HH_2041,POP_2041,AEMP_2041,RTL_2041,IND_2041,OTHR_2041,TPCL_2041,HJI_2041,HH_2042,POP_2042,AEMP_2042,RTL_2042,IND_2042,OTHR_2042,TPCL_2042,HJI_2042,HH_2043,POP_2043,AEMP_2043,RTL_2043,IND_2043,OTHR_2043,TPCL_2043,HJI_2043,HH_2044,POP_2044,AEMP_2044,RTL_2044,IND_2044,OTHR_2044,TPCL_2044,HJI_2044,HH_2045,POP_2045,AEMP_2045,RTL_2045,IND_2045,OTHR_2045,TPCL_2045,HJI_2045,HH_2046,POP_2046,AEMP_2046,RTL_2046,IND_2046,OTHR_2046,TPCL_2046,HJI_2046,HH_2047,POP_2047,AEMP_2047,RTL_2047,IND_2047,OTHR_2047,TPCL_2047,HJI_2047,HH_2048,POP_2048,AEMP_2048,RTL_2048,IND_2048,OTHR_2048,TPCL_2048,HJI_2048,HH_2049,POP_2049,AEMP_2049,RTL_2049,IND_2049,OTHR_2049,TPCL_2049,HJI_2049,HH_2050,POP_2050,AEMP_2050,RTL_2050,IND_2050,OTHR_2050,TPCL_2050,HJI_2050
131,SLCC RSL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,17.0,0.0,0.0,0.0,0.0,0.0,9.0,6.0,22.0,0.0,0.0,0.0,0.0,0.0,11.0,18.0,57.0,0.0,0.0,0.0,0.0,0.0,32.0,33.0,100.0,0.0,0.0,0.0,0.0,0.0,59.0,50.0,153.0,1.0,0.0,0.0,1.0,1.0,91.0,59.0,188.0,1.0,0.0,0.0,1.0,1.0,107.0,66.0,213.0,2.0,0.0,0.0,2.0,2.0,121.0,72.0,231.0,2.0,0.0,0.0,2.0,2.0,132.0,74.0,238.0,2.0,0.0,0.0,2.0,2.0,135.0,78.0,253.0,3.0,0.0,0.0,3.0,3.0,143.0,80.0,260.0,4.0,0.0,0.0,4.0,4.0,148.0,81.0,264.0,4.0,0.0,0.0,4.0,4.0,150.0,82.0,267.0,4.0,0.0,0.0,4.0,4.0,152.0,83.0,268.0,4.0,0.0,0.0,4.0,4.0,153.0,83.0,269.0,5.0,0.0,0.0,5.0,5.0,154.0,83.0,270.0,5.0,0.0,0.0,5.0,5.0,154.0,84.0,271.0,6.0,0.0,0.0,6.0,6.0,157.0,84.0,272.0,6.0,0.0,0.0,6.0,6.0,157.0,84.0,273.0,7.0,0.0,0.0,7.0,7.0,158.0,84.0,274.0,8.0,0.0,0.0,8.0,8.0,159.0,84.0,274.0,9.0,0.0,0.0,9.0,9.0,160.0,84.0,274.0,10.0,0.0,0.0,10.0,10.0,161.0,85.0,274.0,11.0,0.0,0.0,11.0,11.0,164.0,85.0,275.0,12.0,0.0,0.0,12.0,12.0,165.0,85.0,275.0,14.0,0.0,0.0,14.0,14.0,167.0,85.0,275.0,16.0,0.0,0.0,16.0,16.0,169.0,85.0,275.0,18.0,0.0,0.0,18.0,18.0,171.0,86.0,275.0,20.0,0.0,0.0,20.0,20.0,175.0,86.0,275.0,23.0,0.0,0.0,23.0,23.0,178.0,86.0,275.0,26.0,0.0,0.0,26.0,26.0,181.0,86.0,275.0,28.0,0.0,0.0,28.0,28.0,183.0


In [34]:
base.head()

,CenterName,HH_2019,POP_2019,AEMP_2019,RTL_2019,IND_2019,OTHR_2019,TPCL_2019,HJI_2019,HH_2020,POP_2020,AEMP_2020,RTL_2020,IND_2020,OTHR_2020,TPCL_2020,HJI_2020,HH_2021,POP_2021,AEMP_2021,RTL_2021,IND_2021,OTHR_2021,TPCL_2021,HJI_2021,HH_2022,POP_2022,AEMP_2022,RTL_2022,IND_2022,OTHR_2022,TPCL_2022,HJI_2022,HH_2023,POP_2023,AEMP_2023,RTL_2023,IND_2023,OTHR_2023,TPCL_2023,HJI_2023,HH_2024,POP_2024,AEMP_2024,RTL_2024,IND_2024,OTHR_2024,TPCL_2024,HJI_2024,HH_2025,POP_2025,AEMP_2025,RTL_2025,IND_2025,OTHR_2025,TPCL_2025,HJI_2025,HH_2026,POP_2026,AEMP_2026,RTL_2026,IND_2026,OTHR_2026,TPCL_2026,HJI_2026,HH_2027,POP_2027,AEMP_2027,RTL_2027,IND_2027,OTHR_2027,TPCL_2027,HJI_2027,HH_2028,POP_2028,AEMP_2028,RTL_2028,IND_2028,OTHR_2028,TPCL_2028,HJI_2028,HH_2029,POP_2029,AEMP_2029,RTL_2029,IND_2029,OTHR_2029,TPCL_2029,HJI_2029,HH_2030,POP_2030,AEMP_2030,RTL_2030,IND_2030,OTHR_2030,TPCL_2030,HJI_2030,HH_2031,POP_2031,AEMP_2031,RTL_2031,IND_2031,OTHR_2031,TPCL_2031,HJI_2031,HH_2032,POP_2032,AEMP_2032,RTL_2032,IND_2032,OTHR_2032,TPCL_2032,HJI_2032,HH_2033,POP_2033,AEMP_2033,RTL_2033,IND_2033,OTHR_2033,TPCL_2033,HJI_2033,HH_2034,POP_2034,AEMP_2034,RTL_2034,IND_2034,OTHR_2034,TPCL_2034,HJI_2034,HH_2035,POP_2035,AEMP_2035,RTL_2035,IND_2035,OTHR_2035,TPCL_2035,HJI_2035,HH_2036,POP_2036,AEMP_2036,RTL_2036,IND_2036,OTHR_2036,TPCL_2036,HJI_2036,HH_2037,POP_2037,AEMP_2037,RTL_2037,IND_2037,OTHR_2037,TPCL_2037,HJI_2037,HH_2038,POP_2038,AEMP_2038,RTL_2038,IND_2038,OTHR_2038,TPCL_2038,HJI_2038,HH_2039,POP_2039,AEMP_2039,RTL_2039,IND_2039,OTHR_2039,TPCL_2039,HJI_2039,HH_2040,POP_2040,AEMP_2040,RTL_2040,IND_2040,OTHR_2040,TPCL_2040,HJI_2040,HH_2041,POP_2041,AEMP_2041,RTL_2041,IND_2041,OTHR_2041,TPCL_2041,HJI_2041,HH_2042,POP_2042,AEMP_2042,RTL_2042,IND_2042,OTHR_2042,TPCL_2042,HJI_2042,HH_2043,POP_2043,AEMP_2043,RTL_2043,IND_2043,OTHR_2043,TPCL_2043,HJI_2043,HH_2044,POP_2044,AEMP_2044,RTL_2044,IND_2044,OTHR_2044,TPCL_2044,HJI_2044,HH_2045,POP_2045,AEMP_2045,RTL_2045,IND_2045,OTHR_2045,TPCL_2045,HJI_2045,HH_2046,POP_2046,AEMP_2046,RTL_2046,IND_2046,OTHR_2046,TPCL_2046,HJI_2046,HH_2047,POP_2047,AEMP_2047,RTL_2047,IND_2047,OTHR_2047,TPCL_2047,HJI_2047,HH_2048,POP_2048,AEMP_2048,RTL_2048,IND_2048,OTHR_2048,TPCL_2048,HJI_2048,HH_2049,POP_2049,AEMP_2049,RTL_2049,IND_2049,OTHR_2049,TPCL_2049,HJI_2049,HH_2050,POP_2050,AEMP_2050,RTL_2050,IND_2050,OTHR_2050,TPCL_2050,HJI_2050
0,BYU,5.0,20.0,16114.0,6.0,0.0,16108.0,16114.0,16123.0,5.0,20.0,15154.0,6.0,0.0,15148.0,15154.0,15163.0,5.0,21.0,15160.0,6.0,0.0,15154.0,15160.0,15169.0,5.0,21.0,15164.0,6.0,0.0,15158.0,15164.0,15173.0,5.0,21.0,15164.0,6.0,0.0,15158.0,15164.0,15173.0,5.0,21.0,15165.0,6.0,0.0,15159.0,15165.0,15174.0,5.0,21.0,15164.0,6.0,0.0,15158.0,15164.0,15173.0,5.0,21.0,15164.0,6.0,0.0,15158.0,15164.0,15173.0,5.0,21.0,15164.0,6.0,0.0,15158.0,15164.0,15173.0,5.0,23.0,15164.0,6.0,0.0,15158.0,15164.0,15173.0,5.0,22.0,15164.0,6.0,0.0,15158.0,15164.0,15173.0,5.0,23.0,15164.0,6.0,0.0,15158.0,15164.0,15173.0,6.0,24.0,15164.0,6.0,0.0,15158.0,15164.0,15174.8,6.0,24.0,15164.0,6.0,0.0,15158.0,15164.0,15174.8,6.0,25.0,15164.0,6.0,0.0,15158.0,15164.0,15174.8,6.0,26.0,15164.0,6.0,0.0,15158.0,15164.0,15174.8,6.0,26.0,15164.0,6.0,0.0,15158.0,15164.0,15174.8,6.0,30.0,15164.0,6.0,0.0,15158.0,15164.0,15174.8,8.0,33.0,15164.0,6.0,0.0,15158.0,15164.0,15178.4,10.0,35.0,15164.0,6.0,0.0,15158.0,15164.0,15182.0,10.0,36.0,15164.0,6.0,0.0,15158.0,15164.0,15182.0,10.0,37.0,15164.0,6.0,0.0,15158.0,15164.0,15182.0,11.0,38.0,15164.0,6.0,0.0,15158.0,15164.0,15183.8,11.0,40.0,15164.0,6.0,0.0,15158.0,15164.0,15183.8,11.0,40.0,15164.0,6.0,0.0,15158.0,15164.0,15183.8,11.0,41.0,15164.0,6.0,0.0,15158.0,15164.0,15183.8,11.0,44.0,15164.0,6.0,0.0,15158.0,15164.0,15183.8,12.0,44.0,15164.0,6.0,0.0,15158.0,15164.0,15185.6,13.0,45.0,15164.0,6.0,0.0,15158.0,15164.0,15187.4,13.0,45.0,15164.0,6.0,0.0,15158.0,15164.0,15187.4,13.0,47.0,15164.0,6.0,0.0,15158.0,15164.0,15187.4,14.0,52.0,15164.0,6.0,0.0,15158.0,15164.0,15189.2
1,Crossroads,171.0,603.0,928.0,59

In [35]:
base = centers_sdf[['CenterName']].copy()

for pkl in parcel_se_2015:

    year = os.path.splitext(os.path.basename(pkl))[0].split('_')[-1]
    df = pd.read_pickle(pkl)
    
    df['TOTHH'] = df['households']
    df['HHPOP'] = df['hhpop'] 
    df['TPCL'] = df['total_jobs']
    df['ALLEMP'] = df['total_jobs'] # missing mining, home-based, construction 
    df['RETEMP'] = df['retail_jobs']
    df['INDEMP'] = df['industrial_jobs']
    df['OTHEMP'] = df['office_jobs']
    df['HJI'] = df['TOTHH']*1.8 + df['TPCL']
    df = df[['parcel_id', 'TOTHH', 'HHPOP', 'ALLEMP','RETEMP', 'INDEMP', 'OTHEMP', 'TPCL', 'HJI']].copy()
    df.columns = ['parcel_id', f'HH_{year}', f'POP_{year}', f'AEMP_{year}', f'RTL_{year}', f'IND_{year}', f'OTHR_{year}', f'TPCL_{year}', f'HJI_{year}']

    df= df.merge(centers_lu_2015, on='parcel_id', how='left')
    df_by_center = df.groupby('CENTER_NAME').sum().reset_index()
    df_by_center.rename({'CENTER_NAME':'CenterName'}, inplace=True, axis=1)
    del df_by_center['parcel_id']
    base = base.merge(df_by_center, on='CenterName', how='left').copy()

se_by_year_base_2015 = base
se_by_year_base_2015 = se_by_year_base_2015.fillna(0)
se_by_year_base_2015 = se_by_year_base_2015.round()

In [36]:
se_by_year_base_2015.head()

,CenterName,HH_2019,POP_2019,AEMP_2019,RTL_2019,IND_2019,OTHR_2019,TPCL_2019,HJI_2019,HH_2020,POP_2020,AEMP_2020,RTL_2020,IND_2020,OTHR_2020,TPCL_2020,HJI_2020,HH_2021,POP_2021,AEMP_2021,RTL_2021,IND_2021,OTHR_2021,TPCL_2021,HJI_2021,HH_2022,POP_2022,AEMP_2022,RTL_2022,IND_2022,OTHR_2022,TPCL_2022,HJI_2022,HH_2023,POP_2023,AEMP_2023,RTL_2023,IND_2023,OTHR_2023,TPCL_2023,HJI_2023,HH_2024,POP_2024,AEMP_2024,RTL_2024,IND_2024,OTHR_2024,TPCL_2024,HJI_2024,HH_2025,POP_2025,AEMP_2025,RTL_2025,IND_2025,OTHR_2025,TPCL_2025,HJI_2025,HH_2026,POP_2026,AEMP_2026,RTL_2026,IND_2026,OTHR_2026,TPCL_2026,HJI_2026,HH_2027,POP_2027,AEMP_2027,RTL_2027,IND_2027,OTHR_2027,TPCL_2027,HJI_2027,HH_2028,POP_2028,AEMP_2028,RTL_2028,IND_2028,OTHR_2028,TPCL_2028,HJI_2028,HH_2029,POP_2029,AEMP_2029,RTL_2029,IND_2029,OTHR_2029,TPCL_2029,HJI_2029,HH_2030,POP_2030,AEMP_2030,RTL_2030,IND_2030,OTHR_2030,TPCL_2030,HJI_2030,HH_2031,POP_2031,AEMP_2031,RTL_2031,IND_2031,OTHR_2031,TPCL_2031,HJI_2031,HH_2032,POP_2032,AEMP_2032,RTL_2032,IND_2032,OTHR_2032,TPCL_2032,HJI_2032,HH_2033,POP_2033,AEMP_2033,RTL_2033,IND_2033,OTHR_2033,TPCL_2033,HJI_2033,HH_2034,POP_2034,AEMP_2034,RTL_2034,IND_2034,OTHR_2034,TPCL_2034,HJI_2034,HH_2035,POP_2035,AEMP_2035,RTL_2035,IND_2035,OTHR_2035,TPCL_2035,HJI_2035,HH_2036,POP_2036,AEMP_2036,RTL_2036,IND_2036,OTHR_2036,TPCL_2036,HJI_2036,HH_2037,POP_2037,AEMP_2037,RTL_2037,IND_2037,OTHR_2037,TPCL_2037,HJI_2037,HH_2038,POP_2038,AEMP_2038,RTL_2038,IND_2038,OTHR_2038,TPCL_2038,HJI_2038,HH_2039,POP_2039,AEMP_2039,RTL_2039,IND_2039,OTHR_2039,TPCL_2039,HJI_2039,HH_2040,POP_2040,AEMP_2040,RTL_2040,IND_2040,OTHR_2040,TPCL_2040,HJI_2040,HH_2041,POP_2041,AEMP_2041,RTL_2041,IND_2041,OTHR_2041,TPCL_2041,HJI_2041,HH_2042,POP_2042,AEMP_2042,RTL_2042,IND_2042,OTHR_2042,TPCL_2042,HJI_2042,HH_2043,POP_2043,AEMP_2043,RTL_2043,IND_2043,OTHR_2043,TPCL_2043,HJI_2043,HH_2044,POP_2044,AEMP_2044,RTL_2044,IND_2044,OTHR_2044,TPCL_2044,HJI_2044,HH_2045,POP_2045,AEMP_2045,RTL_2045,IND_2045,OTHR_2045,TPCL_2045,HJI_2045,HH_2046,POP_2046,AEMP_2046,RTL_2046,IND_2046,OTHR_2046,TPCL_2046,HJI_2046,HH_2047,POP_2047,AEMP_2047,RTL_2047,IND_2047,OTHR_2047,TPCL_2047,HJI_2047,HH_2048,POP_2048,AEMP_2048,RTL_2048,IND_2048,OTHR_2048,TPCL_2048,HJI_2048,HH_2049,POP_2049,AEMP_2049,RTL_2049,IND_2049,OTHR_2049,TPCL_2049,HJI_2049,HH_2050,POP_2050,AEMP_2050,RTL_2050,IND_2050,OTHR_2050,TPCL_2050,HJI_2050
0,BYU,30.0,140.0,16309.0,52.0,0.0,16257.0,16309.0,16363.0,30.0,140.0,16315.0,52.0,0.0,16263.0,16315.0,16369.0,30.0,140.0,16317.0,52.0,0.0,16265.0,16317.0,16371.0,30.0,140.0,16318.0,52.0,0.0,16266.0,16318.0,16372.0,31.0,142.0,16318.0,52.0,0.0,16266.0,16318.0,16374.0,31.0,142.0,16318.0,52.0,0.0,16266.0,16318.0,16374.0,31.0,142.0,16318.0,52.0,0.0,16266.0,16318.0,16374.0,31.0,142.0,16318.0,52.0,0.0,16266.0,16318.0,16374.0,31.0,142.0,16318.0,52.0,0.0,16266.0,16318.0,16374.0,31.0,142.0,16318.0,52.0,0.0,16266.0,16318.0,16374.0,31.0,142.0,16318.0,52.0,0.0,16266.0,16318.0,16374.0,31.0,142.0,16318.0,52.0,0.0,16266.0,16318.0,16374.0,31.0,142.0,16319.0,53.0,0.0,16266.0,16319.0,16375.0,31.0,142.0,16319.0,53.0,0.0,16266.0,16319.0,16375.0,31.0,142.0,16320.0,53.0,0.0,16267.0,16320.0,16376.0,31.0,142.0,16320.0,53.0,0.0,16267.0,16320.0,16376.0,31.0,142.0,16320.0,53.0,0.0,16267.0,16320.0,16376.0,31.0,142.0,16320.0,53.0,0.0,16267.0,16320.0,16376.0,31.0,142.0,16320.0,53.0,0.0,16267.0,16320.0,16376.0,31.0,142.0,16320.0,53.0,0.0,16267.0,16320.0,16376.0,31.0,142.0,16320.0,53.0,0.0,16267.0,16320.0,16376.0,31.0,142.0,16320.0,53.0,0.0,16267.0,16320.0,16376.0,31.0,142.0,16320.0,53.0,0.0,16267.0,16320.0,16376.0,31.0,142.0,16320.0,53.0,0.0,16267.0,16320.0,16376.0,31.0,142.0,16320.0,53.0,0.0,16267.0,16320.0,16376.0,31.0,142.0,16320.0,53.0,0.0,16267.0,16320.0,16376.0,31.0,142.0,16320.0,53.0,0.0,16267.0,16320.0,16376.0,31.0,142.0,16320.0,53.0,0.0,16267.0,16320.0,16376.0,31.0,142.0,16320.0,53.0,0.0,16267.0,16320.0,16376.0,31.0,142.0,16320.0,53.0,0.0,16267.0,16320.0,16376.0,31.0,142.0,16320.0,53.0,0.0,16267.0,16320.0,16376.0,31

In [37]:
#===============================
# Process CENTER to feature class
#===============================

data_columns = list(se_by_year_base_2019.columns)[1:]

center_gdb = os.path.join(map_folder, 'CENTER.gdb')
if not arcpy.Exists(center_gdb):
    arcpy.CreateFileGDB_management(map_folder, 'CENTER.gdb')

categories = ['HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI']
for c in categories:
    new_cols = [col for col in data_columns if col.split('_')[0] == c]
    new_temp_df = se_by_year_base_2019[['CenterName'] + new_cols].copy()
    old_temp_df = se_by_year_base_2015[['CenterName'] + new_cols].copy()
    

    new_new_names = [col.replace(c, 'N') for col in new_cols]
    new_old_names = [col.replace(c, 'O') for col in new_cols]
    new_rename_dict = dict(zip(new_cols, new_new_names))
    old_rename_dict = dict(zip(new_cols, new_old_names))
    new_temp_df.rename(new_rename_dict, axis=1, inplace=True)
    old_temp_df.rename(old_rename_dict, axis=1, inplace=True)

    merged = new_temp_df.merge(old_temp_df, on='CenterName', how='left')
    merged = centers_sdf.merge(merged, on='CenterName', how='left')

    merged[new_new_names + new_old_names] = merged[new_new_names + new_old_names].fillna(value=0)

    merged.SHAPE.apply(Geometry)
    merged.spatial.set_geometry("SHAPE", inplace=True)
    gsa = GeoSeriesAccessor(merged['SHAPE'])
    merged['ACRES'] = gsa.area * 0.000247105
    merged['ACRES'] = merged['ACRES'].astype(float).round(2)

    outfile = os.path.join(center_gdb, '{}_PROJECTIONS_by_CENTER'.format(c))
    merged.spatial.to_featureclass(location=outfile, sanitize_columns=False)

    # arcpy.AddField_management(outfile, "ACRES", "FLOAT")
    # exp = "round(!SHAPE.AREA@ACRES!,2)"
    # arcpy.CalculateField_management(outfile, "ACRES", exp, "PYTHON3")

In [ ]:
#==========================
# Process CENTER to json
#==========================

chart_CENTER_folder = os.path.join(chart_folder, 'CENTER')
if not os.path.exists(chart_CENTER_folder):
    os.makedirs(chart_CENTER_folder)
else:
    shutil.rmtree(chart_CENTER_folder)
    os.makedirs(chart_CENTER_folder)

new_df = se_by_year_base_2019.copy()
old_df = se_by_year_base_2015.copy()
id_col = 'CenterName'

categories = ('HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI')
cols = [col for col in new_df.columns if col.startswith(categories)]
new_new_cols = ['NEW_' + col for col in cols]
old_new_cols = ['OLD_' + col for col in cols]

new_rename_dict = dict(zip(cols, new_new_cols))
old_rename_dict = dict(zip(cols, old_new_cols))
new_df.rename(new_rename_dict, axis=1, inplace=True)
old_df.rename(old_rename_dict, axis=1, inplace=True)

df = new_df.merge(old_df, on=id_col, how='left')

for index, row in df.iterrows():
        f = open(os.path.join(chart_CENTER_folder, "{}_{}.json".format(id_col, row[id_col])), "a")
        f.write("[\n")
        
        for cat in categories:  
            for year in range(2019,2051):
                
                new_col = '_'.join(['NEW', cat, str(year)])
                old_col = '_'.join(['OLD', cat, str(year)])

                f.write("\t{\n")
                f.write('''\t\t"C":"{}",\n'''.format(cat))
                f.write('''\t\t"Y":{},\n'''.format(year))
                
                f.write('''\t\t"N":{},\n'''.format(row[new_col]))
                f.write('''\t\t"O":{}\n'''.format(row[old_col]))
        
                if cat == "HJI" and year == 2050:
                    f.write("\t}\n")
                else:
                    f.write("\t},\n")

        f.write("]\n")
        f.close()

In [ ]:
def zipdir(path, ziph, ext=None):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        if ext:
            files = [ fi for fi in files if fi.endswith(ext) ]

        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))

with zipfile.ZipFile(os.path.join(map_folder,'CENTER.gdb.zip'),'w', zipfile.ZIP_DEFLATED) as zip_center:
    zipdir(center_gdb, zip_center)